In [110]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px 
import datetime as dt 
import ipywidgets as widgets
from IPython.display import display 
from IPython.display import Markdown as md
import numpy as np
import yfinance as yf

In [111]:
#load data 
dfusa = pd.read_csv('covid-19-data/us.csv')
dfstates = pd.read_csv('covid-19-data/us-states.csv')
dfcounties = pd.read_csv('covid-19-data/us-counties.csv')
dfpop = pd.read_csv('pop_usa_states.csv')

In [112]:
def preprocess(data,s):
    data['date']=pd.to_datetime(data['date'])
    #s is wheter it is data for state or for country s==1: state, s==2:county, s==0: country
    #case for state
    if s==1:
        data.sort_values(['state','date'],inplace=True)
        states = data['state'].unique()
        data['daily_cases'] = np.zeros(data.shape[0])
        data['daily_deaths'] = np.zeros(data.shape[0])
        for s in states:
            data['daily_cases'][data['state']==s] =data.cases[data['state']==s].diff()
            data['daily_deaths'][data['state']==s] =data.deaths[data['state']==s].diff()
        neg_cases = data[data['daily_cases']<0].index
        data.drop(neg_cases, axis=0, inplace=True)
        neg_cases = data[data['daily_deaths']<0].index
        data.drop(neg_cases, axis=0, inplace=True) 
    #case for country 
    elif s==0:
        data['daily_cases'] = data['cases'].diff()
        data['daily_deaths'] = data['deaths'].diff()
        neg_cases = data[data['daily_cases']<0].index
        data.drop(neg_cases, axis=0, inplace=True)
        neg_cases = data[data['daily_deaths']<0].index
        data.drop(neg_cases, axis=0, inplace=True)
        
    else:
        data.sort_values(['county','date'],inplace=True)
        counties = data['county'].unique()
        data['daily_cases'] = data[['county','cases']].groupby(by=["county"]).diff()
        data['daily_deaths'] = data[['county','deaths']].groupby(by=["county"]).diff()
        neg_cases = data[data['daily_cases']<0].index
        data.drop(neg_cases, axis=0, inplace=True)
        neg_cases = data[data['daily_deaths']<0].index
        data.drop(neg_cases, axis=0, inplace=True) 

    
    return data    
    

In [113]:
dfusa = preprocess(dfusa,0)
dfstates= preprocess(dfstates,1)
dfcounties = preprocess(dfcounties,2)

/var/folders/07/cbtj5f0s2_v0hy9qt2t1zqgc0000gn/T/ipykernel_3755/1610212475.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/07/cbtj5f0s2_v0hy9qt2t1zqgc0000gn/T/ipykernel_3755/1610212475.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [114]:
#def to plot the data, if country=0 : plot for the whole country, country =1 plot for states if coountry =2,
#plot for county

def plot(y,data,state='Alabama',county='Autauga County',country=0):
    if country ==0 :
        title = '{} in the USA'.format(y)
        ax = px.area(data, 'date', y, title =title )
        ax.show()
    elif country ==1:
        title = '{} in {}'.format(y,state)
        ax = px.area(data[data['state']==state], 'date',y,title = title)
        ax.show()
    else:
        title = '{} in {}'.format(y,county)
        ax = px.area(data[data['county']==county], 'date',y,title = title)
        ax.show()
        
    

# Insight into covid-19

In [115]:
dropdown_case = widgets.Dropdown(options=['daily_cases','daily_deaths'])
dropdown_country = widgets.Dropdown(options=['country','state','county'])
dropdown_state = widgets.Dropdown(options=dfstates['state'].unique())
dropdown_county =  widgets.Dropdown(options=dfcounties['county'].unique())
input_widgets = widgets.HBox([dropdown_case, dropdown_country])
output = widgets.Output()
def com_filter(case, country, state,county):
    output.clear_output()
    if country == 'country':
        with output:
            plot(case,dfusa,country=0)
    elif country =='state':
        with output:
            display(dropdown_state)
            plot(case,dfstates,state=state, country=1)
    else:
        with output:
            display(dropdown_county)
            plot(case,dfcounties,county=county, country=2)

            

    
    
        
        
def dropdown_case_eventhandler(change):
    com_filter(change.new, dropdown_country.value,dropdown_state.value,dropdown_county.value)

def dropdown_country_eventhandler(change):
    com_filter(dropdown_case.value,change.new,dropdown_state.value,dropdown_county.value)    
    
def dropdown_state_eventhandler(change):
    com_filter(dropdown_case.value,dropdown_country.value,change.new,dropdown_county.value)
def dropdown_county_eventhandler(change):
    com_filter(dropdown_case.value,dropdown_country.value,dropdown_state.value,change.new)


dropdown_case.observe(dropdown_case_eventhandler, names='value')
dropdown_country.observe(dropdown_country_eventhandler, names='value')
dropdown_state.observe(dropdown_state_eventhandler, names='value')
dropdown_county.observe(dropdown_county_eventhandler, names='value')
display(input_widgets)
display(output)
#initial plot
with output:
    display(plot('daily_cases',dfusa,country=0))


Output()

Case fatality ratio (CFR) is the proportion of individuals diagnosed with a disease who die from that disease and is therefore a measure of severity among detected cases

In [116]:
#vaccination started mid december 2020 
#let's take 15 june 2020 
june15 = dfusa[dfusa['date']==dt.datetime.strptime('2020-06-15',"%Y-%m-%d")]
#December 15 2021
dec15 = dfusa[dfusa['date']==dt.datetime.strptime('2021-12-15',"%Y-%m-%d")]
CFRjune15 = float((june15['deaths']/june15['cases']))
CFRdec15 = float((dec15['deaths']/dec15['cases']))
print(CFRjune15,CFRdec15)

0.05464940321851015 0.015918426836646907


## Cause-specific mortality ratio (CSR). This measure is the mortality rate from a specified cause (COVID-19 in our case) for a population.  per state per 1000 ppl

In [117]:
dfpop.rename(columns={"State":"state"},inplace=True)
dfpop.head()

,rank,state,Pop,Growth,Pop2021,Pop2010,growthSince2010,Percent,density
0,1,California,39664128,0.0013,39613493,37319502,0.0628,0.1180,254.6179
1,2,Texas,30097526,0.0124,29730311,25241971,0.1924,0.0896,115.2138
2,3,Florida,22177997,0.0106,21944577,18845537,0.1768,0.0660,413.5757
3,4,New York,19223191,-0.0040,19299981,19399878,-0.0091,0.0572,407.9105
4,5,Pennsylvania,12805190,0.0001,12804123,12711160,0.0074,0.0381,286.1943


In [118]:
dfstates = pd.merge(dfstates, dfpop[['state','Pop']], how="left", on="state")

In [148]:
dfstates.head()

,date,state,fips,cases,deaths,daily_cases,daily_deaths,Pop
0,2020-03-13,Alabama,1,6,0,NaN,NaN,4949697.0
1,2020-03-14,Alabama,1,12,0,6.0,0.0,4949697.0
2,2020-03-15,Alabama,1,23,0,11.0,0.0,4949697.0
3,2020-03-16,Alabama,1,29,0,6.0,0.0,4949697.0
4,2020-03-17,Alabama,1,39,0,10.0,0.0,4949697.0


In [220]:
#compute csr 
def comp_csr(data,date_end,state=True):
    if state:
        date = dt.datetime.strptime(date_end,"%Y-%m-%d")
        csr={}
        data = data.dropna()
        states = data['state'].unique()
        for s in states:
            time = data[(data['date']==date)&(data['state']==s)]
            rate = (time['deaths'].values[0]/time['Pop'].values[0])*1000
            csr[s]=rate
        title = 'Cause-specific mortality ratio per 1000 people per state on {}'.format(date_end)
        ax = px.bar(x = csr.keys(), y=csr.values(), title=title)
        ax.show()
    else :
        date = dt.datetime.strptime(date_end,"%Y-%m-%d")
        pop = 329500000 #us population in 2020 
        time = data[data['date']==date]
        rate = (time['deaths'].values[0]/pop)*1000
        return round(rate,3)
    
    
    
def comp_cfr(data,date_end,state=True):
# mucfr = death(t)/infected(t) *100 
    if state:
        data = data.dropna()
        states = data['state'].unique()
        date = dt.datetime.strptime(date_end,"%Y-%m-%d")
        cfr={}
        for s in states:
            time = data[(data['date']==date)&(data['state']==s)]
            rate = (time['deaths'].values[0]/time['cases'].values[0])*100
            cfr[s]=rate
        title = 'Case to fatality ratio in percent per state on {}'.format(date_end)
        ax = px.bar(x = cfr.keys(), y=cfr.values(), title=title)
        ax.show()
    else:
        date = dt.datetime.strptime(date_end,"%Y-%m-%d")
        time = data[data['date']==date]
        rate = (time['deaths'].values[0]/time['cases'].values[0])*100
        return round(rate,3) 

In [221]:
date = '2020-12-15'
print('Cause-specific mortality ratio per 1000 people in the US on {}:'.format(date),
      comp_csr(dfusa,date,state=False), 'Per 1000 people')
comp_csr(dfstates,date)


print('Case to fatality ratio in percent in the US on {}:'.format(date),
      comp_cfr(dfusa,date,state=False),'%')
comp_cfr(dfstates,date)


Cause-specific mortality ratio per 1000 people in the US on 2020-12-15: 0.923 Per 1000 people


Case to fatality ratio in percent in the US on 2020-12-15: 1.813 %


In [222]:
date = '2022-05-15'
print('Cause-specific mortality ratio per 1000 people in the US on {}:'.format(date),
      comp_csr(dfusa,date,state=False),'per 1000 people')
print('Case to fatality ratio in percent in the US on {}:'.format(date),
      comp_cfr(dfusa,date,state=False),'%')

Cause-specific mortality ratio per 1000 people in the US on 2022-05-15: 3.03 per 1000 people
Case to fatality ratio in percent in the US on 2022-05-15: 1.211 %


We can see that even tho the cause-specific mortality ratio increased since the begining of the vaccination campaign, the case to fatality decreased of 0.5%. Thus since people have been getting vaccinated there are less covid related deaths

In [93]:
#state with the highest total cases, deaths 
h_c_state = dfstates[dfstates['cases']==dfstates['cases'].max()]
date = h_c_state['date'].dt.strftime('%Y-%m-%d').values[0]
state = h_c_state['state'].values[0]
cases = h_c_state['cases'].values[0]
h_d_state = dfstates[dfstates['deaths']==dfstates['deaths'].max()]
date_d = h_d_state['date'].dt.strftime('%Y-%m-%d').values[0]
state_d = h_d_state['state'].values[0]
deaths = h_c_state['deaths'].values[0]
print('The state that has the most cases as of {} is {} with a total of {} cases.'.format(date,state,cases))
print('The state that has the most deaths as of {} is {} with a total of {} deaths.'.format(date_d,state_d,deaths))

The state that has the most cases as of 2022-05-23 is California with a total of 9461626 cases.
The state that has the most deaths as of 2022-05-23 is California with a total of 91231 deaths.


In [230]:
last_upd = dfusa.iloc[-1]
tot_cases = last_upd['cases']
tot_deaths = last_upd['deaths']
date = last_upd['date'].strftime('%Y-%m-%d')
print('As of {}, {} people were tested positive and {} people died of COVID-19 in the US.'.format(date,tot_cases,tot_deaths))

As of 2022-05-23, 83325014 people were tested positive and 1000826 people died of COVID-19 in the US


# Economic effect of Covid-19 on Nasdaq and S&P500

In [112]:
sp500='^GSPC'
nasdaq = '^NDX'
start_date = dfusa['date'].min()
end_date = dfusa['date'].max()


In [230]:
ticker = yf.Ticker(sp500)
dfsp500 = ticker.history(start=start_date, end=end_date)
dfsp500['name'] = 'sp500'
dfsp500.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,name
Date,,,,,,,,
2020-01-21,3321.030029,3329.790039,3316.610107,3320.790039,4105340000,0,0,sp500
2020-01-22,3330.020020,3337.770020,3320.040039,3321.750000,3619850000,0,0,sp500
2020-01-23,3315.770020,3326.879883,3301.870117,3325.540039,3764860000,0,0,sp500
2020-01-24,3333.100098,3333.179932,3281.530029,3295.469971,3707130000,0,0,sp500
2020-01-27,3247.159912,3258.850098,3234.500000,3243.629883,3823100000,0,0,sp500


In [231]:
ticker = yf.Ticker(nasdaq)
dfndx = ticker.history(start=start_date, end=end_date)
dfndx['name'] = 'nasdaq'
dfndx.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,name
Date,,,,,,,,
2020-01-21,9145.639648,9187.419922,9141.780273,9166.629883,2684540000,0,0,nasdaq
2020-01-22,9212.349609,9242.480469,9179.269531,9188.580078,2449390000,0,0,nasdaq
2020-01-23,9186.139648,9221.129883,9145.230469,9216.980469,2460050000,0,0,nasdaq
2020-01-24,9266.349609,9272.370117,9101.440430,9141.469727,2611710000,0,0,nasdaq
2020-01-27,8913.160156,9001.440430,8910.969727,8952.179688,2583330000,0,0,nasdaq


In [239]:
dfstocks = dfsp500.append(dfndx)

In [240]:
max_y =dfstocks['Close'].max()
fig = px.line(dfstocks,y= 'Close' ,color='name')
fig.add_vline(x='2020-03-13',line_dash="dash") #emergency state in the USA
fig.add_vline(x='2020-12-14',line_dash="dash") #first vaccine in the USA
fig.add_vline(x='2021-06-01',line_dash="dash") #delta variant
fig.add_vline(x='2021-11-26',line_dash="dash") #omicron variant
fig.add_annotation(x='2020-03-13', y=max_y,
            text="Emergency state in the USA",
            showarrow=True,
            arrowhead=1)
fig.add_annotation(x='2020-12-14', y=max_y,
            text="First vaccin in the USA",
            showarrow=True,
            arrowhead=1)
fig.add_annotation(x='2021-06-01', y=max_y,
            text="Delta variant",
            showarrow=True,
            arrowhead=1)
fig.add_annotation(x='2021-11-26', y=max_y,
            text="Omircron variant",
            showarrow=True,
            arrowhead=1)




In [210]:
#compute changes over 7 days in % after each of those events
def changes(data,events):
    changes= list()
    for e in events:
        e_start = data['Close'][(data.index ==e)].values[0]
        e_end_d = dt.datetime.strptime(e, "%Y-%m-%d")+ dt.timedelta(days=7)
        e_end = data['Close'][(data.index ==e_end_d)].values[0]
        e_change=((e_end-e_start)/e_start)*100
        changes.append(e_change)
    return changes

In [233]:
ndx_changes = changes(dfndx,['2020-03-13','2020-12-14','2021-06-01','2021-11-26'])
sp500_changes = changes(dfsp500,['2020-03-13','2020-12-14','2021-06-01','2021-11-26'])

In [238]:
md("#### The declaration of the emergency state had a big impact on both s&p500 and nasdaq stock as their values were down by  {}% and {}% respectively. Neither the first vaccin nor the arrival of the delta variant seem to have had an impact   as both stayed on the same trend. The Omicron variant has a small impact on both as their values were down by {}% and {}% after 7 days of the first case."
   .format(round(ndx_changes[0],3),round(sp500_changes[0],3), round(ndx_changes[3],2),round(sp500_changes[3],2)))

#### The declaration of the emergency state had a big impact on both s&p500 and nasdaq stock as their values were down by  -12.52% and -14.98% respectively. Neither the first vaccin nor the arrival of the delta variant seem to have had an impact   as both stayed on the same trend. The Omicron variant has a small impact on both as their values were down by -1.96% and -1.22% after 7 days of the first case.

In [18]:
##anova for prisons 
pri = pd.read_csv('covid-19-data/prisons/systems.csv')

In [45]:
pri.drop([50,51,52,53],inplace=True)

KeyError: '[50 51 52 53] not found in axis'

In [54]:
pri.head()

,system,inmate_tests,total_inmate_cases,total_inmate_deaths,latest_inmate_population,max_inmate_population_2020,total_officer_cases,total_officer_deaths,total_cases,total_deaths
0,Alabama,15505.0,1601,64,19144.0,21900.0,1019.0,3.0,2620.0,67.0
1,Alaska,27756.0,2428,5,5011.0,5143.0,85.0,0.0,2513.0,5.0
2,Arizona,43652.0,12229,44,38248.0,41674.0,2739.0,0.0,14968.0,44.0
3,Arkansas,0.0,11378,52,14729.0,15886.0,424.0,5.0,11802.0,57.0
4,California,121207.0,49511,217,110471.0,117767.0,16090.0,26.0,65601.0,243.0


In [47]:
pri.describe()

,inmate_tests,total_inmate_cases,total_inmate_deaths,latest_inmate_population,max_inmate_population_2020,total_officer_cases,total_officer_deaths,total_cases,total_deaths
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,76442.720000,6902.400000,45.900000,20678.420000,23632.580000,1956.840000,3.480000,8859.240000,49.380000
std,130952.898986,8859.677962,54.334283,25759.930956,29959.091043,2892.597113,7.220266,11583.795269,59.809524
min,0.000000,169.000000,0.000000,0.000000,1397.000000,45.000000,0.000000,214.000000,0.000000
25%,12442.250000,2103.750000,9.750000,5466.250000,6053.750000,405.250000,0.000000,2536.750000,11.250000
50%,29985.000000,4099.000000,31.000000,13731.000000,15323.500000,1018.500000,1.500000,5573.500000,32.000000
75%,73595.000000,8483.750000,52.750000,24962.250000,30990.750000,2158.250000,4.000000,11288.250000,55.500000
max,673822.000000,49511.000000,220.000000,127814.000000,162000.000000,16090.000000,44.000000,65601.000000,250.000000


In [48]:
pri['total_cases'] = pri['total_inmate_cases']+pri['total_officer_cases']
pri['total_deaths'] = pri['total_inmate_deaths']+pri['total_officer_deaths']


In [55]:
#case fatality ratio 
pri['case_fat_r_in'] =(pri['total_inmate_deaths']/pri['total_inmate_cases'])*100
pri['case_fat_r_of'] =(pri['total_officer_deaths']/pri['total_officer_cases'])*100

In [56]:
pri.head()

,system,inmate_tests,total_inmate_cases,total_inmate_deaths,latest_inmate_population,max_inmate_population_2020,total_officer_cases,total_officer_deaths,total_cases,total_deaths,case_fat_r_in,case_fat_r_of
0,Alabama,15505.0,1601,64,19144.0,21900.0,1019.0,3.0,2620.0,67.0,3.997502,0.294406
1,Alaska,27756.0,2428,5,5011.0,5143.0,85.0,0.0,2513.0,5.0,0.205931,0.000000
2,Arizona,43652.0,12229,44,38248.0,41674.0,2739.0,0.0,14968.0,44.0,0.359800,0.000000
3,Arkansas,0.0,11378,52,14729.0,15886.0,424.0,5.0,11802.0,57.0,0.457022,1.179245
4,California,121207.0,49511,217,110471.0,117767.0,16090.0,26.0,65601.0,243.0,0.438286,0.161591
